# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

H. Beuther  ->  H. Beuther  |  ['H. Beuther']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
A. P  ->  A. P. Garcia  |  ['A. P']
Arxiv has 73 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.12892


extracting tarball to tmp_2510.12892...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2510.12892/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authors.tex' from 'tmp_2510.12892/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

extracting tarball to tmp_2510.13509...

 done.


Found 66 bibliographic references in tmp_2510.13509/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2510.13556


extracting tarball to tmp_2510.13556...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.12892-b31b1b.svg)](https://arxiv.org/abs/2510.12892) | **ALMAGAL VIII. Cataloging Hierarchical Mass Structure from Cores to Clumps across the Galactic Disk**  |
|| J. Wallace, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2025-10-16*|
|*Comments*| *29 pages, 12 figures, submitted to the Astrophysical Journal*|
|**Abstract**|            Investigating the multi-scale fragmentation of dense clumps into compact cores is essential for understanding the processes that govern the initial distribution of mass in stellar clusters and how high-mass stars ($>8~M_{\odot}$) form. We present a catalog of the hierarchical continuum structure from 904 clumps observed in the ALMAGAL program, a high resolution ($0.15-0.8$\arcsec) 1.38 mm Atacama Large Millimeter/submillimeter Array (ALMA) large program targeting dense clumps capable of high-mass star formation throughout the Galactic disk. We use \verb|astrodendro|, a dendrogram-based algorithm, on a uniform linear resolution (2000 au) version of the data to extract 5160 continuum structures with effective radii spanning $800-42000$ au and estimated masses between $~0.05-670~M_{\odot}$. With our large sample, we statistically examine the difference in clump properties for regions with varying levels of hierarchical complexity. We find that clumps exhibiting the richest hierarchical morphology have distributions with higher dust temperatures, surface densities, luminosity-to-mass (\textit{L/M}) ratios, and most massive core (MMC) masses, indicating that these regions tend to be at later evolutionary stages. We find a positive correlation between the mass of cores from the ALMAGAL core catalog and the surface density of their surrounding structures identified in this work. However, this correlation is weaker for cores in more evolved clumps, where lower mass cores can be found at higher local surface densities. This could indicate that some cores accrete mass less efficiently from the intra-clump reservoir than others, despite the total available mass increasing over time, a scenario that is congruent with a clump-fed core accretion model.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.13509-b31b1b.svg)](https://arxiv.org/abs/2510.13509) | **Euclid: Exploring observational systematics in cluster cosmology -- a comprehensive analysis of cluster counts and clustering**  |
|| A. Fumagalli, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-10-16*|
|*Comments*| *16 pages, 14 figures*|
|**Abstract**|            This study explores the impact of observational and modelling systematic effects on cluster number counts and cluster clustering and provides model prescriptions for their joint analysis, in the context of the \Euclid survey. Using 1000 \Euclid-like cluster catalogues, we investigate the effect of systematic uncertainties on cluster summary statistics and their auto- and cross-covariance, and perform a likelihood analysis to evaluate their impact on cosmological constraints, with a focus on the matter density parameter $\Omega_{\rm m}$ and on the power spectrum amplitude $\sigma_8$. Combining cluster clustering with number counts significantly improves cosmological constraints, with the figure of merit increasing by over 300\% compared to number counts alone. We confirm that the two probes are uncorrelated, and the cosmological constraints derived from their combination are almost insensitive to the cosmology dependence of the covariance. We find that photometric redshift uncertainties broaden cosmological posteriors by 20--30\%, while secondary effects like redshift-space distortions (RSDs) have a smaller impact on the posteriors -- 5\% for clustering alone, 10\% when combining probes -- but can significantly bias the constraints if neglected. We show that clustering data below $60\,h^{-1}\,$Mpc provides additional constraining power, while scales larger than acoustic oscillation scale add almost no information on $\Omega_{\rm m}$ and $\sigma_8$ parameters. RSDs and photo-$z$ uncertainties also influence the number count covariance, with a significant impact, of about 15--20\%, on the parameter constraints.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.13556-b31b1b.svg)](https://arxiv.org/abs/2510.13556) | **The 2024 outburst of the neutron star LMXB EXO 0748-676: an investigation of bursts and eclipses with AstroSat**  |
|| <mark>A. P</mark>, et al. |
|*Appeared on*| *2025-10-16*|
|*Comments*| *32 pages, 12 figures, 5 tables*|
|**Abstract**|            We present a detailed analysis of the Type-I (thermonuclear) X-ray bursts and eclipses observed from the neutron star low-mass X-ray binary (LMXB) EXO 0748--676 with AstroSat during the second known outburst of the source following a 16-year-long quiescence period. We detect three thermonuclear X-ray bursts, with two displaying simultaneous coverage in the soft X-rays. Simultaneous UV observations show evidence of reprocessed burst emissions in the far-ultraviolet band. The time-resolved spectral analysis reveals the photospheric radius expansion (PRE) nature for two bursts. We estimate the distance to the source to be $7.42\pm0.53$ kpc using the peak flux of PRE. Notably, one of the bursts exhibited a secondary peak, $\sim30$ s after the primary, particularly dominating in the softer X-rays, which reveals a correlation with the evolution of burst hotspot radius with no temperature dependence. The burst properties and corresponding flux values suggest that mixed H/He burning may have fueled the bursts. We also detect evidence of a soft excess during one burst, likely arising from the interaction of the burst photons with the surroundings. We uncover evidence for a hard X-ray deficit during the peak of all bursts and a hard lag of $\sim4$ s, which can be attributed to the Compton cooling of the corona by the burst photons. We also probe the temporal evolution and the energy dependence of the eclipses, which offer insights into the binary environment. Our study helps gain deeper insight into the physics of burst ignition, flame propagation, the burst-accretion interaction, and the evolution of LMXBs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.12892/./ALMAGAL_masssigma_mmcmass_update.png', 'tmp_2510.12892/./ALMAGAL_masssigma_mcore_evstages_update.png', 'tmp_2510.12892/./ALMAGAL_MvR_leafbranch_full_catalog.png', 'tmp_2510.12892/./ALMAGAL_MvR_RIS_catalogs.png', '', '']
copying  tmp_2510.12892/./ALMAGAL_masssigma_mmcmass_update.png to _build/html/
copying  tmp_2510.12892/./ALMAGAL_masssigma_mcore_evstages_update.png to _build/html/
copying  tmp_2510.12892/./ALMAGAL_MvR_leafbranch_full_catalog.png to _build/html/
copying  tmp_2510.12892/./ALMAGAL_MvR_RIS_catalogs.png to _build/html/
file not found 
file not found 
exported in  _build/html/2510.12892.md
    + _build/html/tmp_2510.12892/./ALMAGAL_masssigma_mmcmass_update.png
    + _build/html/tmp_2510.12892/./ALMAGAL_masssigma_mcore_evstages_update.png
    + _build/html/tmp_2510.12892/./ALMAGAL_MvR_leafbranch_full_catalog.png
    + _build/html/tmp_2510.12892/./ALMAGAL_MvR_RIS_catalogs.png
    + _build/html/
    + _build/html/
found figures ['tmp_2510.13509

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\GG}[1]$</div>



<div id="title">

# ALMAGAL VIII. Cataloging Hierarchical Mass Structure from Cores to Clumps\ across the Galactic Disk

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.12892-b31b1b.svg)](https://arxiv.org/abs/2510.12892)<mark>Appeared on: 2025-10-16</mark> -  _29 pages, 12 figures, submitted to the Astrophysical Journal_

</div>
<div id="authors">

J. Wallace, et al. -- incl., <mark>H. Beuther</mark>

</div>
<div id="abstract">

**Abstract:** Investigating the multi-scale fragmentation of dense clumps into compact cores is essential for understanding the processes that govern the initial distribution of mass in stellar clusters and how high-mass stars ( $>8 M_{\odot}$ ) form. We present a catalog of the hierarchical continuum structure from 904 clumps observed in the ALMAGAL program, a high resolution ( $0.15-0.8$ $\arcsec$ ) 1.38 mm Atacama Large Millimeter/submillimeter Array (ALMA) large program targeting dense clumps capable of high-mass star formation throughout the Galactic disk. We use $\verb$ |astrodendro|, a dendrogram-based algorithm, on a uniform linear resolution (2000 au) version of the data to extract 5160 continuum structures with effective radii spanning $800-42000$ au and estimated masses between $ 0.05-670 M_{\odot}$ . With our large sample, we statistically examine the difference in clump properties for regions with varying levels of hierarchical complexity. We find that clumps exhibiting the richest hierarchical morphology have distributions with higher dust temperatures, surface densities, luminosity-to-mass ( _L/M_ ) ratios, and most massive core (MMC) masses, indicating that these regions tend to be at later evolutionary stages. We find a positive correlation between the mass of cores from the ALMAGAL core catalog and the surface density of their surrounding structures identified in this work. However, this correlation is weaker for cores in more evolved clumps, where lower mass cores can be found at higher local surface densities. This could indicate that some cores accrete mass less efficiently from the intra-clump reservoir than others, despite the total available mass increasing over time, a scenario that is congruent with a clump-fed core accretion model.

</div>

<div id="div_fig1">

<img src="tmp_2510.12892/./ALMAGAL_masssigma_mmcmass_update.png" alt="Fig9.1" width="50%"/><img src="tmp_2510.12892/./ALMAGAL_masssigma_mcore_evstages_update.png" alt="Fig9.2" width="50%"/>

**Figure 9. -** (Left panel) A 2D kernel density estimate plot of the mass of the MMC as a function of the surface density of its most immediate underlying dendrogram structure that has an effective radius at least 3 times the radius of the core ($\Sigma_{\text{surr}}$). (Right panel) The same figure, but we include all cores from the ALMAGAL core catalog. ALMAGAL targets are divided into three evolutionary categories based on their _L/M_ ratio.  Contours indicate the shape of the distribution for the inner [90, 70, 40]\% of the data.  Marginal plots indicate the 1D kernel density estimates of the dendrogram structure surface density and core mass on the _x_- and _y_-axes, respectively.  (*fig:surfd_mcore*)

</div>
<div id="div_fig2">

<img src="tmp_2510.12892/./ALMAGAL_MvR_leafbranch_full_catalog.png" alt="Fig8.1" width="50%"/><img src="tmp_2510.12892/./ALMAGAL_MvR_RIS_catalogs.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** (Left) A 2D Kernel density estimate plot of the mass-radius relation for leaf (green) and branch structures (red) in the ALMAGAL hierarchical structure catalog. Best fits corresponding to the power law relation $M \propto R^{\alpha}$ for each population are indicated with the solid and dashed black lines. (Right) A 2D kernel density estimate plot for the mass-radius relation for all dendrogram structures in the Rich (orange), Simple (green), and Isolated (purple) sub-samples. For both panels, the contours indicate the shape of the distributions for the inner [90, 50, 20]\% of the data (*fig:mvr_full*)

</div>
<div id="div_fig3">

<img src="" alt="Fig1.1" width="50%"/><img src="" alt="Fig1.2" width="50%"/>

**Figure 1. -** The maximum dendrogram level distribution for the near (top panel) and far (bottom panel) clumps in our ALMAGAL sample. The vertical dashed lines indicate the 1$\sigma_{\text{level}}$ and 3$\sigma_{\text{level}}$ of the fitted exponential distribution, shown by the solid black line. Since we cannot use the exact 1$\sigma_{\text{level}}$ and 3$\sigma_{\text{level}}$ values as thresholds on the integer-valued dendrogram levels, we instead round each value to the nearest integer and use that as a threshold. The integer ranges for the maximum level are shown for the Rich, Simple, and Isolated targets. The Empty targets do not have a threshold value, since they represent regions with non-detections and do not have an associated dendrogram level. (*fig:max_lvl_dists*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.12892"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\af}[1]{\textcolor{red1}{AF:~#1}}$
$\newcommand{\todo}[1]{\textcolor{red1}{TO DO:~#1}}$
$\newcommand{\comm}[1]{\textcolor{blue1}{#1}}$
$\newcommand{\orcid}[1]$
$\newcommand{\om}{\Omega_{\rm m}}$
$\newcommand{\logas}{\logten(A_{\rm s})}$
$\newcommand{\lnas}{\ln(10^{10} A_s)}$
$\newcommand{\mwl}{{\rm M}_{\rm WL}}$
$\newcommand{\lob}{\lambda^{\rm ob}}$
$\newcommand{\ltr}{\lambda^{\rm tr}}$
$\newcommand{\zob}{z^{\rm ob}}$
$\newcommand{\ztr}{z^{\rm tr}}$
$\newcommand{\mmin}{M_{\rm min}}$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# $\Euclid$: Exploring systematics effects in cluster cosmology -- a comprehensive analysis of cluster counts and clustering$\thanks{This paper is published on behalf of the Euclid Consortium.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.13509-b31b1b.svg)](https://arxiv.org/abs/2510.13509)<mark>Appeared on: 2025-10-16</mark> -  _16 pages, 14 figures_

</div>
<div id="authors">

A. Fumagalli, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** This study explores the impact of observational and modelling systematic effects on cluster number counts and cluster clustering and provides model prescriptions for their joint analysis, in the context of the $\Euclid$ survey.   Using 1000 $\Euclid$ -like cluster catalogues, we investigate the effect of systematic uncertainties on cluster summary statistics and their auto- and cross-covariance, and perform a likelihood analysis to evaluate their impact on cosmological constraints, with a focus on the matter density parameter $\om$ and on the power spectrum amplitude $\sigma_8$ .   Combining cluster clustering with number counts significantly improves cosmological constraints, with the figure of merit increasing by over 300 \% compared to number counts alone. We confirm that the two probes are uncorrelated, and the cosmological constraints derived from their combination are almost insensitive to the cosmology dependence of the covariance. We find that photometric redshift uncertainties broaden cosmological posteriors by 20--30 \% , while secondary effects like redshift-space distortions (RSDs) have a smaller impact on the posteriors -- 5 \% for clustering alone, 10 \% when combining probes -- but can significantly bias the constraints if neglected. We show that clustering data below $60 h^{-1} $ Mpc provides additional constraining power, while scales larger than acoustic oscillation scale add almost no information on $\om$ and $\sigma_8$ parameters. RSDs and photo- $z$ uncertainties also influence the number count covariance, with a significant impact, of about 15--20 \% , on the parameter constraints.

</div>

<div id="div_fig1">

<img src="tmp_2510.13509/./images/cov_diff.png" alt="Fig3" width="100%"/>

**Figure 3. -** Residuals of the clustering covariance diagonal terms for the lowest (blue) and highest (red) redshift bins and $\lambda \in [20, 30]$. The grey area marks the 10\% region. _Top_: Residuals of the distorted numerical covariance relative to the undistorted redshift case. Solid lines indicate the RSDs case, while dashed lines include both RSDs and photo-$z$ effects ($\sigma_{z0} = 0.01$). _Bottom_: Residuals between analytical and numerical covariances for different cases: solid lines for true redshift, dashed for RSDs, and dotted for both RSDs and photo-$z$ effects. (*fig:cl_cov_diff*)

</div>
<div id="div_fig2">

<img src="tmp_2510.13509/./images/NC_cov_RSD.png" alt="Fig6" width="100%"/>

**Figure 6. -** Number count covariance terms:  numerical matrix (shaded areas, representing the 1$\sigma$ region), analytical model with power spectrum monopole (solid lines), and model with effective power spectrum (see Eq. \ref{eq:eff_pk}, dashed lines). Colour-coded terms represent different components: grey for diagonal shot-noise, blue for diagonal sample variance, red and yellow for first and second off-diagonal sample covariance between mass bins, respectively, and green for first off-diagonal sample covariance in redshift bins.  (*fig:nc_cov_RSD*)

</div>
<div id="div_fig3">

<img src="tmp_2510.13509/./images/cross_cov.png" alt="Fig12.1" width="50%"/><img src="tmp_2510.13509/./images/cross_logl.png" alt="Fig12.2" width="50%"/>

**Figure 12. -** Cross-covariance between number counts and clustering. For better visualisation, here we use redshift bins of width $\Delta z=0.2$ for counts and $\Delta z=0.5$ for the 2PCF. _Left_: Auto- and cross-correlation matrix of number counts and clustering, computed from 1000 mocks. _Right_: Log-likelihood residuals for number counts and clustering, for each one of the 1000 lightcones, with respect to the mean value assuming the fiducial model parameters. (*fig:cross_cov*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.13509"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

121  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
